In [1]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division
import tensorflow as tf
import numpy as np
import scipy.misc
import scipy
import random
from tensorflow.examples.tutorials.mnist import input_data

sess = tf.InteractiveSession()

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

conv1_act = np.load('./data_rep/Conv1_Activations_5000_good.npy')

C = 1
M = 16
N = 32

#print(conv1_act.shape)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [ ]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def residuals(x, mnist, C, M, N):
    a = tf.Variable(tf.truncated_normal([N,M], stddev=0.1), name="a")
    s = tf.Variable(tf.truncated_normal([5,5,1,M], stddev=0.1), name="basis")
    b = tf.Variable(tf.constant(0.1, shape=[N]), name="bias")
    w = []
    for i in range(N):
        temp_w = []
        for j in range(M):
            temp_w.append(a[i,j]*s[:,:,:,j])
        w.append(tf.reshape(tf.add_n(temp_w), [5,5,1,1]))
    w = tf.concat(3,w)
    x_image = tf.reshape(x, [-1,28,28,1])
    conv1_pred = tf.nn.relu(conv2d(x_image, w) + b)
    return conv1_pred,s 

x = tf.placeholder(tf.float32, shape=[None, 784])
output = tf.placeholder(tf.float32, [None, None, None, None])

pred,s = residuals(x, mnist, C, M, N)

diff = (output - pred)

l2_loss = tf.nn.l2_loss(diff)/10.0 #+ 0.001*tf.reduce_sum(tf.abs(s))

train_step = tf.train.AdamOptimizer(1e-4).minimize(l2_loss)

epochs = 2000

sess.run(tf.global_variables_initializer())
#print(np.mean(l2_loss.eval(feed_dict={x:mnist.train.images[10:20], output:conv1_act[10:20,:]})))
#print(np.mean(conv1_act))
for i in range(epochs):
    for j in range(500):
        start_idx = j*10
        end_idx = (j + 1)*10
        train_step.run(feed_dict={x:mnist.train.images[start_idx:end_idx], output:conv1_act[start_idx:end_idx,:]})
    print("Total Loss at epoch {}: ".format(i), 
          l2_loss.eval(feed_dict={x:mnist.train.images[:5000], output:conv1_act}))
    start_ridx = np.random.randint(0,high=4000)
    end_ridx = start_ridx + 10
    print("Random Loss at epoch {}: ".format(i), 
          l2_loss.eval(feed_dict={x:mnist.train.images[start_ridx:end_ridx], output:conv1_act[start_ridx:end_ridx,:]}))
    #print(np.sum(pred.eval(feed_dict={x:mnist.train.images[:5000], output:conv1_act})))

Total Loss at epoch 0:  inf
Random Loss at epoch 0:  43.5118
Total Loss at epoch 1:  inf
Random Loss at epoch 1:  16.315
Total Loss at epoch 2:  inf
Random Loss at epoch 2:  7.68099
Total Loss at epoch 3:  inf
Random Loss at epoch 3:  5.39189
Total Loss at epoch 4:  inf
Random Loss at epoch 4:  3.58342
Total Loss at epoch 5:  inf
Random Loss at epoch 5:  3.27933
Total Loss at epoch 6:  inf
Random Loss at epoch 6:  3.31723
Total Loss at epoch 7:  inf
Random Loss at epoch 7:  2.25819
Total Loss at epoch 8:  inf
Random Loss at epoch 8:  2.94452
Total Loss at epoch 9:  inf
Random Loss at epoch 9:  2.13086
Total Loss at epoch 10:  inf
Random Loss at epoch 10:  1.58604
Total Loss at epoch 11:  inf
Random Loss at epoch 11:  2.12448
Total Loss at epoch 12:  inf
Random Loss at epoch 12:  1.83808
Total Loss at epoch 13:  inf
Random Loss at epoch 13:  1.68655
Total Loss at epoch 14:  inf
Random Loss at epoch 14:  1.58672
Total Loss at epoch 15:  inf
Random Loss at epoch 15:  1.46189
Total Loss at

In [ ]:
saver = tf.train.Saver()
saver.save(sess, "MNIST_conv1_reconstruct_2k.ckpt")

In [ ]:
residuals(xf.x, mnist, conv1_act)

In [ ]:
xf.message

In [ ]:
residuals(x0, mnist, conv1_act)